In [1]:
import os, sys
sys.path.append(os.getcwd())

import time
import pickle
import numpy as np
import tensorflow as tf
import tflib.plot
import utils
import tflib as lib
import tflib.linear
import tflib.conv1d
import models
from random import shuffle

In [2]:
class VirtualArgparse:
    
    # Path to lstm and english words

    training_data = "final_lstm_op.txt"
    
    # Name of directory to output
    output_dir = "pretrained"
    
    save_every = 100  
    iters = 20000   
    batch_size = 64
    seq_length = 10
    layer_dim = 128
    critic_iters = 10
    lamb = 10

In [3]:
args = VirtualArgparse

In [4]:
if not os.path.isdir(args.output_dir):
    os.makedirs(args.output_dir)

if not os.path.isdir(os.path.join(args.output_dir, 'checkpoint')):
    os.makedirs(os.path.join(args.output_dir, 'checkpoint'))

if not os.path.isdir(os.path.join(args.output_dir, 'samples')):
    os.makedirs(os.path.join(args.output_dir, 'samples'))

In [5]:
#load the dataset
lines, charmp, inv_charmp = utils.load_dataset(
    path=args.training_data,
    max_length=args.seq_length)

loaded 774592 lines in dataset


In [6]:
#store the charmap weigths
with open(os.path.join(args.output_dir, 'charmap.pickle'), 'wb') as f:
    pickle.dump(charmp, f)

with open(os.path.join(args.output_dir, 'charmap_inv.pickle'), 'wb') as f:
    pickle.dump(inv_charmp, f)
    
print("Number of unique characters in dataset: {}".format(len(charmp)))

Number of unique characters in dataset: 124


In [7]:

real_inputs_discrete = tf.placeholder(tf.int32, shape=[args.batch_size, args.seq_length])
real_inputs = tf.one_hot(real_inputs_discrete, len(charmp))

In [8]:
#inputs to discriminator
print(real_inputs_discrete)
print(real_inputs)

Tensor("Placeholder:0", shape=(64, 10), dtype=int32)
Tensor("one_hot:0", shape=(64, 10, 124), dtype=float32)


In [9]:
#input random noise to generator
fake_inputs = models.Generator(args.batch_size, args.seq_length, args.layer_dim, len(charmp))
fake_inputs_discrete = tf.argmax(fake_inputs, fake_inputs.get_shape().ndims-1)

W1212 00:01:54.253783  3860 deprecation_wrapper.py:119] From C:\Users\Piyush's PC\DLproject_2019\models.py:64: The name tf.random_normal is deprecated. Please use tf.random.normal instead.



In [10]:
print(fake_inputs)

Tensor("Reshape_2:0", shape=(64, 10, 124), dtype=float32)


In [11]:
#discriminator real and fake 
disc_real = models.Discriminator(real_inputs, args.seq_length, args.layer_dim, len(charmp))
disc_fake = models.Discriminator(fake_inputs, args.seq_length, args.layer_dim, len(charmp))

discriminator_cost = tf.reduce_mean(disc_fake) - tf.reduce_mean(disc_real)
generator_cost = -tf.reduce_mean(disc_fake)

In [12]:
# WGAN lipschitz-penalty
alpha = tf.random_uniform(
    shape=[args.batch_size,1,1],
    minval=0.,
    maxval=1.
)

differences = fake_inputs - real_inputs
interpolates = real_inputs + (alpha*differences)
gradients = tf.gradients(models.Discriminator(interpolates, args.seq_length, args.layer_dim, len(charmp)), [interpolates])[0]
slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1,2]))
gradient_penalty = tf.reduce_mean((slopes-1.)**2)
discriminator_cost += args.lamb * gradient_penalty

gen_params = lib.params_with_name('Generator')
disc_params = lib.params_with_name('Discriminator')

gen_train_op = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5, beta2=0.9).minimize(generator_cost, var_list=gen_params)
disc_train_op = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5, beta2=0.9).minimize(discriminator_cost, var_list=disc_params)

W1212 00:01:58.103499  3860 deprecation.py:323] From C:\Users\installations\lib\site-packages\tensorflow\python\ops\math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [13]:
# Dataset iterator
def inf_train_gen():
    while True:
        np.random.shuffle(lines)
        for i in range(0, len(lines)-args.batch_size+1, args.batch_size):
            yield np.array(
                [[charmp[c] for c in l] for l in lines[i:i+args.batch_size]],
                dtype='int32'
            )

In [14]:
#calculate js divergence
true_char_ngram = [utils.NgramLanguageModel(i+1, lines[10*args.batch_size:], tokenize=False) for i in range(4)]
validation_char_ngram = [utils.NgramLanguageModel(i+1, lines[:10*args.batch_size], tokenize=False) for i in range(4)]
for i in range(4):
    print("validation set JSD for n={}: {}".format(i+1, true_char_ngram[i].js_with(validation_char_ngram[i])))
true_char_ngram = [utils.NgramLanguageModel(i+1, lines, tokenize=False) for i in range(4)]

validation set JSD for n=1: 0.004985971679623904
validation set JSD for n=2: 0.0667152892948728
validation set JSD for n=3: 0.3281906483195437
validation set JSD for n=4: 0.6475340885547007


In [15]:
#training usually takes longer time for every 100 steps
with tf.Session() as session:

    # Time stamp
    localtime = time.asctime( time.localtime(time.time()) )
    print("Starting TensorFlow session...")
    print("Local current time :", localtime)
    
    # Start TensorFlow session...
    session.run(tf.global_variables_initializer())

    def generate_samples():
        samples = session.run(fake_inputs)
        samples = np.argmax(samples, axis=2)
        decoded_samples = []
        for i in range(len(samples)):
            decoded = []
            for j in range(len(samples[i])):
                decoded.append(inv_charmp[samples[i][j]])
            decoded_samples.append(tuple(decoded))
        return decoded_samples

    gen = inf_train_gen()

    for iteration in range(args.iters + 1):
        
        start_time = time.time()
        
        # Training the Generator
        if iteration > 0:
            _ = session.run(gen_train_op)
        
        
        # Training the Discriminator
        for i in range(args.critic_iters):
            _data = next(gen)
            
            _gen_cost, _ = session.run(
                [generator_cost,gen_train_op],
            feed_dict={fake_inputs_discrete:_data}
            
            )
                
            _disc_cost, _ = session.run(
                [discriminator_cost, disc_train_op],
                feed_dict={real_inputs_discrete:_data }
            
            )
          
       
        lib.plot.output_dir = args.output_dir
        lib.plot.plot('time', time.time() - start_time)
        lib.plot.plot('train generator cost', _gen_cost)
        
        
        lib.plot.plot_generator('time', time.time() - start_time)
        lib.plot.plot_generator('train discriminator cost', _disc_cost)

        if iteration % 100 == 0 and iteration > 0:

            samples = []
            for i in range(10):
                samples.extend(generate_samples())

            for i in range(4):
                lm = utils.NgramLanguageModel(i+1, samples, tokenize=False)
                lib.plot.plot('js{}'.format(i+1), lm.js_with(true_char_ngram[i]))

            with open(os.path.join(args.output_dir, 'samples', 'samples_{}.txt').format(iteration), 'w') as f:
                for s in samples:
                    s = "".join(s)
                    f.write(s + "\n")
                    
                    
        if iteration % args.save_every == 0 and iteration > 0:
            model_saver = tf.train.Saver()
            model_saver.save(session, os.path.join(args.output_dir, 'checkpoint', 'checkpoint_{}.ckpt').format(iteration))
            print("{} / {} ({}%)".format(iteration, args.iters, iteration/args.iters*100.0 ))

        if iteration == args.iters:
            print("...Training done.")
            
           
        
        if iteration % 100 == 0:
            
            lib.plot.flush_generator()   
            lib.plot.flush()
        
        lib.plot.tick()
        
        
            
# Time stamp calculation
localtime = time.asctime( time.localtime(time.time()) )
print("Ending TensorFlow session.")
print("Local current time :", localtime)

Starting TensorFlow session...
Local current time : Thu Dec 12 00:02:53 2019


KeyboardInterrupt: 